In [1]:
%pylab inline
figsize(8,8)
import sys
import rootpath
sys.path.append(rootpath.detect())
from scipy.spatial import distance_matrix

Populating the interactive namespace from numpy and matplotlib


In [2]:
def weak_dominates(Y, x):
    """
    Test whether rows of Y weakly dominate x
    
    Parameters
    ----------
    Y : array_like
        Array of points to be tested. 
        
    x : array_like
        Vector to be tested
        
    Returns
    -------
    c : ndarray (Bool)
        1d-array.  The ith element is True if Y[i] weakly dominates x
    """
    return (Y <= x ).sum(axis=1) == Y.shape[1]


def attainment_sample(Y, Nsamples=1000):
    """
    Return samples from the attainment surface defined by the mutually non-dominating set Y

    Parameters
    ---------
    Y : array_like
        The surface to be sampled. Each row of Y is vector, that is mutually
        with all the other rows of Y
    Nsamples : int
        Number of samples

    Returns
    -------
    S : ndarray
        Array of samples from the attainment surface.
        Shape; Nsamples by Y.shape[1] 
    
    Notes
    -----
    See "Dominance-based multi-objective simulated annealing"
    Kevin Smith, Richard Everson, Jonathan Fieldsend, 
    Chris Murphy, Rashmi Misra.
    IEEE Transactions on Evolutionary Computing. 
    Volume: 12, Issue: 3, June 2008.
    https://ieeexplore.ieee.org/abstract/document/4358782
    """
    N, D = Y.shape
    Ymin = Y.min(axis=0)
    r = Y.max(axis=0) - Ymin
    S = np.zeros((Nsamples, D))
    
    # Set up arrays of the points sorted according to each coordinate.
    Ys = np.zeros((N, D))
    for d in range(D):
        Ys[:,d] = np.sort(Y[:,d])

    for n in range(Nsamples):
        v = np.random.rand(D)*r + Ymin
        m = np.random.randint(D)

        # Bisection search to find the smallest v[m] 
        # so that v is weakly dominated by an element of Y
        lo, hi = 0, N
        while lo < hi:
            mid = (lo+hi)//2
            v[m] = Ys[mid,m]
            if not any(weak_dominates(Y, v)):
                lo = mid+1
            else:
                hi = mid
        if lo == N: lo -= 1
        v[m] = Ys[lo, m]      
        assert lo == N-1 or any(weak_dominates(Y, v))
        S[n,:] = v[:]
    return S


In [3]:
def normalise_to_axes(x, axes=None):
    r = 1
    assert x.ndim == 2
    axes = np.ones(x.shape[1]) if axis is None else np.array(axes)
    
    x_norm = np.zeros_like(x)
    for i, xi in enumerate(x):
        lmbda = sqrt(r**2/np.sum([xi[j]**2/axes[j]**2 for j in range(x.shape[1])]))
        x_norm[i] = xi*lmbda
        
    return x_norm

In [4]:
n_points = 1000
a = 4
b = 2
c = 1

random normal

In [5]:
%matplotlib qt

In [1]:
fig = figure()

ax1 = fig.add_subplot(2,1,2, projection="3d")
ax1.view_init(elev=35., azim=65)
# first subfigure: multivariate norm projected to elipsoid
ax0 = fig.add_subplot(2,1,1, projection="3d", sharex=ax1, sharey=ax1, sharez=ax1)
ax0.view_init(elev=35., azim=65)
ax0.set_box_aspect([a,b,c]) 
ax0.set_title("Projection of distribution $\sigma_a=6$, $\sigma_b=1$ \t to elipse: $a=6$, $b=1$")
x2 = abs(random.multivariate_normal([0, 0, 0], np.diag([a**2, b**2, c**2]), size=10000))
x2_norm = normalise_to_axes(x2, [a, b, c])
ax0.scatter(*x2_norm[:n_points].T, c="C1", alpha=0.4, s=8, label = "points projected to elipse".format(n_points))
ax0.scatter([0],[0], [0], c="C3", label="origin")
ax0.legend()

# second subfigure: attainment surface sample based on subfigure 1 samples
ax1.set_box_aspect([a,b,c]) 
ax1.set_title("Attainment surface samples")
x3 = attainment_sample(x2_norm, n_points*2)
min_d = distance_matrix(x2_norm, x3).min(axis=0)

ax1.scatter(*x3[min_d<0.05][:n_points].T, c="C1", alpha=0.4, s=8, label = "attainment sample".format(n_points))
ax1.scatter([0],[0], [0], c="C3", label="origin")

ax1.legend()

NameError: name 'figure' is not defined

In [8]:
print(sum(min_d<0.05))
print(n_points)

1650
1000
